# Create Knowledge Assistant Brick

This notebook demonstrates how to create a Knowledge Assistant brick using the AgentBricksManager class.
The Knowledge Assistant will use the financial documents uploaded to our Unity Catalog volume.

In [ ]:
# Import configuration and helper functions
%run ../config.py
%run ../resources/brick_setup_functions

import sys
import logging
from databricks.sdk import WorkspaceClient

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s",
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

In [ ]:
# Initialize Workspace Client and AgentBricksManager
w = WorkspaceClient()
brick_manager = AgentBricksManager(w)

print(f"Using catalog: {catalog}")
print(f"Using schema: {schema}")
print(f"Using volume: {volume_name}")

In [ ]:
# Define the volume path for our uploaded documents
volume_path = f"/Volumes/{catalog}/{schema}/{volume_name}"
print(f"Volume path: {volume_path}")

# Define knowledge sources - one for each subdirectory in /data
# This creates separate knowledge sources for better document organization
knowledge_sources = [
    {
        "files_source": {
            "name": "quarterly_reports_10q",
            "type": "files",
            "description": "10-Q quarterly financial reports from Apple, Meta, Amazon, Google, NVIDIA, Microsoft, and Tesla",
            "files": {"path": f"{volume_path}/10Q"}
        }
    },
    {
        "files_source": {
            "name": "annual_reports_10k",
            "type": "files",
            "description": "10-K annual financial reports and comprehensive business overviews",
            "files": {"path": f"{volume_path}/10k"}
        }
    },
    {
        "files_source": {
            "name": "earnings_releases",
            "type": "files",
            "description": "Quarterly earnings releases, press releases, and financial announcements",
            "files": {"path": f"{volume_path}/earning_releases"}
        }
    },
    {
        "files_source": {
            "name": "annual_reports",
            "type": "files",
            "description": "Annual reports and comprehensive business reviews",
            "files": {"path": f"{volume_path}/annual_reports"}
        }
    },
    {
        "files_source": {
            "name": "earnings_transcripts",
            "type": "files",
            "description": "Earnings call transcripts and Q&A sessions with management",
            "files": {"path": f"{volume_path}/earning_transcripts"}
        }
    }
]

print(f"Configured {len(knowledge_sources)} knowledge source(s):")
for i, source in enumerate(knowledge_sources):
    source_info = source['files_source']
    print(f"  {i+1}. {source_info['name']}: {source_info['description']}")
    print(f"      Path: {source_info['files']['path']}")

In [ ]:
# Create the Knowledge Assistant
ka_name = "Financial_Analysis_Assistant"
ka_description = "A Knowledge Assistant for analyzing financial documents and providing insights on company performance, earnings, and financial metrics."
ka_instructions = """You are a financial analysis assistant. Use the provided financial documents to answer questions about:
- Company financial performance and trends
- Earnings analysis and key metrics
- Risk factors and business outlook
- Quarterly and annual report insights

Always cite specific documents and sections when providing analysis. Be precise with numbers and dates."""

print(f"Creating Knowledge Assistant: {ka_name}")
print(f"Description: {ka_description}")
print("\nInstructions:")
print(ka_instructions)

In [ ]:
# Create or update the Knowledge Assistant using the AgentBricksManager
try:
    result = brick_manager.ka_create_or_update(
        name=ka_name,
        knowledge_sources=knowledge_sources,
        description=ka_description,
        instructions=ka_instructions
    )
    
    print("\n✅ Knowledge Assistant created/updated successfully!")
    
    # Extract key information from the result
    tile_id = result['knowledge_assistant']['tile']['tile_id']
    operation = result.get('operation', 'unknown')
    
    print(f"Operation: {operation}")
    print(f"Tile ID: {tile_id}")
    print(f"Name: {result['knowledge_assistant']['tile']['name']}")
    
    # Check status
    status = brick_manager.ka_get_endpoint_status(tile_id)
    print(f"Status: {status}")
    
    # Display knowledge sources
    sources = result.get('knowledge_assistant', {}).get('knowledge_sources', [])
    print(f"\nKnowledge Sources ({len(sources)}):")
    for i, source in enumerate(sources):
        print(f"  {i+1}. {source.get('name', 'Unknown')} - {source.get('description', 'No description')}")
        
except Exception as e:
    print(f"❌ Error creating Knowledge Assistant: {e}")
    raise

In [ ]:
# Wait for the Knowledge Assistant to be ready (if it's still provisioning)
print("\n🔄 Checking Knowledge Assistant status...")

current_status = brick_manager.ka_get_endpoint_status(tile_id)
print(f"Current status: {current_status}")

if current_status in ['PROVISIONING', 'NOT_READY']:
    print("Knowledge Assistant is still provisioning. You can check its status in the Databricks workspace.")
    print(f"Navigate to Machine Learning > Knowledge Assistants to view the assistant with ID: {tile_id}")
elif current_status == 'ONLINE':
    print("✅ Knowledge Assistant is ready to use!")
    print(f"You can test it in the Databricks workspace or use the tile_id: {tile_id}")
else:
    print(f"Knowledge Assistant status: {current_status}")

In [ ]:
# Add some example questions to help users understand what the KA can do
example_questions = [
    {
        "question": "What were the key revenue trends in the most recent quarterly report?",
        "guideline": "Focus on revenue growth, segment performance, and year-over-year comparisons"
    },
    {
        "question": "What are the main risk factors mentioned in the 10-K filing?",
        "guideline": "Identify and summarize the primary business and financial risks disclosed"
    },
    {
        "question": "How did earnings per share change compared to the previous quarter?",
        "guideline": "Provide specific EPS figures and calculate percentage changes"
    },
    {
        "question": "What guidance did management provide for the upcoming quarter or year?",
        "guideline": "Extract forward-looking statements and management guidance from earnings calls or reports"
    }
]

print(f"\n📚 Adding {len(example_questions)} example questions...")

try:
    # Add examples in batch for efficiency
    created_examples = brick_manager.ka_add_examples_batch(tile_id, example_questions)
    print(f"✅ Successfully added {len(created_examples)} example questions")
    
    for i, example in enumerate(created_examples):
        question_text = example.get('question', 'Unknown question')
        print(f"  {i+1}. {question_text[:60]}{'...' if len(question_text) > 60 else ''}")
        
except Exception as e:
    print(f"⚠️ Warning: Could not add example questions: {e}")
    print("The Knowledge Assistant was created successfully, but examples failed to add.")

In [ ]:
# Summary and next steps
print("\n🎉 Knowledge Assistant Setup Complete!")
print("\n📋 Summary:")
print(f"  • Name: {ka_name}")
print(f"  • Tile ID: {tile_id}")
print(f"  • Status: {brick_manager.ka_get_endpoint_status(tile_id)}")
print(f"  • Knowledge Sources: {len(knowledge_sources)} configured")
print(f"  • Volume Path: {volume_path}")

print("\n🚀 Next Steps:")
print("1. Wait for the Knowledge Assistant to be ONLINE if it's still provisioning")
print("2. Test the assistant in the Databricks workspace (Machine Learning > Knowledge Assistants)")
print("3. Try asking questions about financial performance, earnings, and company metrics")
print("4. Use the example questions provided to get started")

print(f"\n🔗 Access your Knowledge Assistant:")
print(f"   Navigate to Machine Learning > Knowledge Assistants in your Databricks workspace")
print(f"   Look for: {ka_name}")